<a href="https://colab.research.google.com/github/VasireddyNandini/predictive-analysis/blob/prod_pred/prod_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# Step 1: Install required libraries (if needed)
# pip install pandas scikit-learn

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Step 2: Sample purchase history data
data = {
    'user_id': ['U1', 'U1', 'U2', 'U2', 'U3', 'U3', 'U4', 'U4', 'U5', 'U5'],
    'product_id': ['P1', 'P2', 'P2', 'P3', 'P1', 'P4', 'P3', 'P4', 'P1', 'P5'],
    'purchase': [1]*10  # Binary purchase history (1 = bought)
}

df = pd.DataFrame(data)

# Step 3: Create user-product matrix
user_product_matrix = df.pivot_table(index='user_id', columns='product_id', values='purchase', fill_value=0)

# Step 4: Calculate item-to-item similarity
product_similarity = cosine_similarity(user_product_matrix.T)
product_similarity_df = pd.DataFrame(product_similarity, index=user_product_matrix.columns, columns=user_product_matrix.columns)

# Step 5: Recommend products based on last purchase
def recommend_products(user_id, top_n=3):
    user_row = user_product_matrix.loc[user_id]
    purchased_products = user_row[user_row > 0].index.tolist()

    scores = {}
    for product in purchased_products:
        similar_products = product_similarity_df[product].drop(index=product)
        for similar_product, score in similar_products.items():
            if user_row[similar_product] == 0:  # Not already purchased
                scores[similar_product] = scores.get(similar_product, 0) + score

    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    recommended = [product for product, score in sorted_scores][:top_n]

    return recommended

# Step 6: Predict for user U2
user = 'U2'
recommendations = recommend_products(user)

print(f"Recommended products for user {user}: {recommendations}")


Recommended products for user U2: ['P4', 'P1', 'P5']
